In [ ]:
import os
import time 
import numpy as np 
import torch
import random
import matplotlib.pyplot as plt
from torch.utils import data
from loader.QSM_data_loader2 import QSM_data_loader2
from utils.data import *
from utils.files import *
from models.unet import Unet

In [ ]:
? python

In [ ]:
import nibabel

In [ ]:
GPU = 2

In [ ]:
if GPU == 1:
    FD_DATA = '/home/sdc/Jinwei/QSM/1*1*3recon/ms1' # GPU 1
    rootDir = '/home/sdc/Jinwei/BayesianQSM'
elif GPU == 2:
    FD_DATA = '/data/Jinwei/Bayesian_QSM/20190523_BayesianQSM_dataset/1_1_3recon/ms2' # GPU 2
    rootDir = '/data/Jinwei/Bayesian_QSM/weight'
    
factor = 3.9034
filename = '{0}/RDF.mat'.format(FD_DATA)
RDF = np.real(load_mat(filename, varname='RDF'))/factor

filename = '{0}/Mask.mat'.format(FD_DATA)
Mask = np.real(load_mat(filename, varname='Mask'))

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
unet3d = Unet(input_channels=1, output_channels=2, num_filters=[2**i for i in range(5, 10)])
unet3d.to(device)
unet3d.load_state_dict(torch.load(rootDir+'/weights.pt'))
unet3d.eval()

In [ ]:
RDF = torch.tensor(RDF[np.newaxis, np.newaxis, ...], device=device).float()
outputs = unet3d(RDF)

In [ ]:
pred_mean = outputs[0, 0, ...]
pred_var = outputs[0, 1, ...]

In [ ]:
pred_mean = np.asarray(pred_mean.cpu().detach())
pred_var = np.asarray(pred_var.cpu().detach())

In [ ]:
idx = 25

plt.imshow(pred_mean[:,:,idx]*Mask[:,:,idx], cmap='Greys_r', clim=(-0.15,0.15))
plt.axis('off')
plt.show()

plt.imshow(np.exp(pred_var[:,:,idx]/2)*Mask[:,:,idx], cmap='Greys_r')
plt.axis('off')
plt.show()